# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [1]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图呈现了一个宁静美丽的自然景观。图中的主要特点是一条木制的栈道，它穿越在郁郁葱葱的草地上，引向远处看不到的终点。草地上覆盖着茂密的绿色植被，显示出生机勃勃的春夏季节。背景中天空呈现出澄清的蓝色，点缀着几朵轻盈的白云，增添了一种宁静和平和的氛围。\n\n整个场景光线充足，阳光和蓝天与绿色植被的对比令人感觉清新愉快。这样的环境可能是理想的徒步旅行地点，提供了亲近自然和放松心情的完美机会。整体上，这幅图传达了大自然的宁静与和谐，是观赏和体验自然美景的一种', role='assistant', function_call=None, tool_calls=None))


In [2]:
response.choices[0].message.content

'这幅图呈现了一个宁静美丽的自然景观。图中的主要特点是一条木制的栈道，它穿越在郁郁葱葱的草地上，引向远处看不到的终点。草地上覆盖着茂密的绿色植被，显示出生机勃勃的春夏季节。背景中天空呈现出澄清的蓝色，点缀着几朵轻盈的白云，增添了一种宁静和平和的氛围。\n\n整个场景光线充足，阳光和蓝天与绿色植被的对比令人感觉清新愉快。这样的环境可能是理想的徒步旅行地点，提供了亲近自然和放松心情的完美机会。整体上，这幅图传达了大自然的宁静与和谐，是观赏和体验自然美景的一种'

### 封装成一个函数 query_image_description

In [20]:
def query_image_description(url, prompt="介绍下这幅图?"):
    # client = OpenAI()  # 初始化 OpenAI 客户端
    from openai import OpenAI
    client = OpenAI(base_url = 'https://api.xiaoai.plus/v1', api_key = 'sk-BBdn99z7cZuhVylF6fE5BdAf7735479d9bE4854f8a161067')
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [21]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图是一种幽默的比较，用来形象化地表达16岁和工作后的状态对比。左边的图显示了一只体型魁梧，肌肉发达的狗，代表16岁时候的活力与朝气；右边的图则是一只显得有些憔悴和无力的狗，象征工作后的疲惫状态。图中每一只狗旁边都有一些描述性文字，分别陈述这两个阶段的特点，如左边的狗标签有“我刚做一片沙果，身体会疼痛”，这些都是幽默夸张的描述，用以增强对比效果。整体而言，这幅图通过夸张和幽默的方式，表达了从年轻到职业生涯过程中可能遇到的变化和挑战。


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [11]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI(base_url="https://api.xiaoai.plus/v1", api_key="sk-20kg2ng944ZYhmdr87Db79E9323e48DaA1470c40B4B5439c")
# client = OpenAI()  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.xiaoai.plus/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [13]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

这幅图显示了从1980年到2020年间美国、中国、日本和德国的国内生产总值（GDP）比较。从图中可以看出：

1. **美国（蓝色线条）**：美国GDP在这40年间持续增长，以稳定的趋势逐年上升，是所有国家中GDP最高的。

2. **中国（红色线条）**：中国的GDP从1980年的相对较低水平开始，显示出显著的增长，尤其是从2000年以后增长非常迅速，到2020年接近美国的GDP。

3. **日本（紫色线条）**：日本的GDP在1990年代初达到高峰后，进入了长期的停滞期，表现为波动但总体趋势平稳。

4. **德国（绿色线条）**：德国的GDP增长较为稳定，虽然增速没有中国那样显著，但整体保持了持续的增长。

此图有效地展示了这四个经济体在过去四十年的经济表现，特别是中国的快速崛起以及日本的经济停滞情况。


#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [14]:
content = query_base64_image_description("./images/handwriting_0.jpg")
print(content)

这张图片展示的是一本笔记本上的笔记，内容主要关于人工智能或者机器学习领域的一些技术，具体涉及到参数微调方法如Prompt Tuning和相关优化技术，比如LoRA（Low-Rank Adaptation）。

1. **Prompt Tuning（提示调整）**：提及使用小型模型进行Prompt Tuning的情况，这种方法涉及到对输入数据X进行embedding嵌入转换，通过一定的编码器或解码器来处理这些数据。

2. **Prefix Tuning**：这部分内容介绍了在Transformer模型中，如何通过前缀调整来优化模型性能。

3. **LoRA的使用和优势**:
   - 通过修改权重矩阵W，使用ΔW = AB的方式，其中A和B是低秩矩阵，这样通过低秩近似来减少需要训练的参数数量，这对于减少计算资源消耗和提高效率非常有帮助。
   - 展示了使用Q-LORA和LAMA的示例，显示了使用LoRA的参数数量减少了（从78GB减少到4GB），表明使用低秩替代能显著减小模型大小。

这些笔记显然是用于学术学习或研发工作中的要点记录，帮助理解和实施当前AI领域的一些高级技术。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [11]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这张图片显示的是一本笔记本上手写的文字，内容主要涉及到自然语言处理（NLP）中的训练技术，如Prompt Tuning和LoRA技术。

1. **Prompt Tuning（简要模型调整）**： 提到了使用Prompt Tuning来调整一个小型的Transformer模型。此处解释了输入X（由个别输入X1, X2, ..., Xn组成）。每个输入首先通过一个Embedding过程转换，然后通过Token变换。输出Y是通过矩阵W与转换后的输入X'之间的乘法得出。

2. **Prefix Tuning：** 这部分说明了Prefix Tuning的过程，其中添加了前缀权重W_p到原始权重W_j中，得到新的权重W'用于生成输出Y。

3. **LoRA调整技术**： 这部分涉及Linear Re-parameterization（线性重新参数化），通过调整矩阵ΔW（通过两个矩阵A和B的乘积表示）来修改权重W。这是一种节省参数调整的方法，使原有模型的W变为W+ΔW，这里也涉及到了一些矩阵运算和优化策略。

其中还提到了两个案例分析的存储需求：“LLAMA”需要65GB，而经过LoRA调整的“QLLoRA”仅需要48GB。

这些笔记对于理解NLP中一些先进的模型调整技术十分有用，尤其对于需要在资源受限的环境下部署NLP模型的研究人员或实践者。

![](./images/handwriting_1.jpg)

In [12]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

这张图片展示的是一本笔记本的两页，内容涉及深度学习、特别是关于自然语言处理（NLP）的各种技术和方法。主要讨论了Transformer模型及其改进方法和训练技术。

左侧页面的上部标注有“自然语言处理”、“基础”和“评价”，可能是对内容的分类。提到了Transformer模型，并列举了不同的测试标准和指标，如PeFT (“Prompt-based Fine-Tuning”) 和模型性能对比（“Benchmark”）。此外，还提到了不同的方法，如Prompt Tuning和Adapter。具体包括：  
- Adapter: 一个2019年Google的研究
- Prefix: 代表2021年Stanford的工作
- Prompt: 同样是2021年Google的研究
- P-Tuning V1和V2：2021年的两种方法
- Soft prompts：2021年的研究，提示模板基于模板

右侧页面讨论了多模态指令式微调（multi-modality instruction FT）、Llama (3B)、LoRA、PETC（2022年的新技术）等。还有部分文字描述了如何使用prefix-tuning和Adapter方法来细化在大型语言模型（LLMs）中的处理。

页面提到了几种语言模型，如：
- Llama 
- BLOOM
- ChatGLM 
- Alpaca

这些内容表明这本笔记本的主人正在研究或学习NLP领域的最新技术和方法，特别是如何通过各种微调技术提升已有的大型语言模型的性能。

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。

In [4]:
from openai import OpenAI
# client = OpenAI(base_url="https://api.xiaoai.plus/v1", api_key="sk-20kg2ng944ZYhmdr87Db79E9323e48DaA1470c40B4B5439c")
client = OpenAI(base_url = 'https://api.xiaoai.plus/v1', api_key = 'sk-BBdn99z7cZuhVylF6fE5BdAf7735479d9bE4854f8a161067')

In [17]:
content = query_base64_image_description("./images/letter_02.jpg")
print(content)

这张图片展示的是一首手写的英语诗歌。诗歌的书写风格是非常优雅的书法，每一个字符的笔迹都流畅且具有艺术感。诗歌的标题是“Accomplish’d Virgin”。

诗中描述了一位贞洁无暇的少女，她的生活充满了虔诚和纯洁的梦想。她从晨光中醒来，带着一种坚定的美德去面对新的一天。这位少女把自己的任务执行得非常早，就连“晓星”奥罗拉（常指黎明女神）看到她时都会脸红。她的活力和热情被比喻为希腊智慧女神帕拉斯的眼睛，充满了羡慕。诗中还提到，她那让人惊叹的帐篷和那些散发着活力的人物形象，不仅美丽而且能教人以道德为指导。

整体来说，这首诗歌欣赏和颂扬了这位少女的纯洁和美德，通过美丽的比喻和形象生动的描写，传达了对其敬仰和赞美的情感。同时，每一行的结构和韵律都非常讲究，展示了诗人深厚的文学功底和对语言的敏感把握。


In [18]:
from IPython.display import display, Markdown

content = query_base64_image_description("./images/letter_02.jpg")
display(Markdown(content))

这张图片展示的是一段用英文手写的诗句。这段诗稿书写得非常工整美观，采用的是一种连笔式的书法字体。诗句内容具有一定的古典韵味，描述的是一位被赞扬的处女（Virgo），她清晨起来进行祈祷，她的纯洁和美德连晨曦（Aurora）都为之一震。诗中还包含了对于智慧女神帕拉斯（Pallas）的引用，她似乎在羡慕着这位处女的纯洁与忙碌。诗句中也涉及了爱神（Love），提到了爱神也许渴望领略她的美丽和清纯。最后几行诗句描绘了一幅通过刺绣讲述故事的画面，每一针每一线中仿佛都包含着某种道德寓意。

具体来说，诗句强调了通过传统的女性美德和艺术形式（如刺绣）传递的教育和道德价值。整体反映了对于女性美德和工艺技能的赞美。

In [19]:
content = query_base64_image_description("./images/notes.jpg")
display(Markdown(content))

这张图片展示了一些关于微分方程的笔记和解法，包括解析和具体的例子。内容可以分为几个部分：

1. **一阶微分方程的解法**：
   - **变量分离法**：形式为 \( \frac{dy}{dx} = g(x) \cdot h(y) \)，通过变量分离和积分求解。
   - **齐次方程**：形式为 \( \frac{dy}{dx} = f(\frac{y}{x}) \)，通过变量变换 \( y = ux \) 来解决。
   - **线性方程**：形式为 \( \frac{dy}{dx} + p(x)y = q(x) \)，使用积分因子方法解决。

2. **高阶微分方程**：
   - 示例包括二阶微分方程 \( y'' = f(x,y') \) 和 \( y''= f(y,y') \)，通过变量代换降阶处理。
   - 讨论了二阶齐次线性方程 \( y'' + p(x)y' + q(x)y = 0 \) 的特解形式，如指数型解 \( y = e^{rx} \)。

3. **微分方程的应用和分类**：
   - 包括了法向量、流形、以及解的存在性和唯一性。
   - 提及了不同类型的方程和解法，例如：
     - \( y''+p(x)y'+q(x)y=f(x) \) 类型的非齐次方程的求解。
     - 使用特解和通解的表述。

4. **常见的微分方程形式和解法**：
   - 给定特定函数形式 \( f(x) = Ru(x) \) 时的解的表达式。
   - 如何处理含有 \( f(x) = e^{ax} \sin bx \) 或 \( e^{ax} \cos bx \) 类型的方程。

这些笔记显示了基本的微分方程处理技巧和一些标准解法，适用于数学或工程学领域中解决实际问题。

In [23]:
notes = query_base64_image_description("./images/notes.jpg", "扩展下图片里面的内容")
display(Markdown(notes))

这幅图像包含的内容主要涉及高等数学中的常微分方程的解法。从图中可看到几种不同类型的微分方程及相应的解题方法和公式推导，以下是对图像中内容的扩展和解释：

1. **第一部分**：
   - **分离变量法**：适用于形式为 dy/dx = g(x)*h(y) 的微分方程。通过重新整理形式并积分求解。
   - **一阶线性微分方程**：形式为 dy/dx + P(x)y = Q(x)。解决方法通常涉及求解积分因子 \( \mu(x) = e^{\int P(x) \, dx} \)。
   - **全微分方程**：如果一个方程可以写成全微分形式，即 \( M(x, y)dx + N(x, y)dy = 0 \) 是恰当的，则可以找到一个函数 \( \Psi(x,y) \) 使 \( d\Psi = 0 \)。

2. **常微分方程的一般解法**：
   - 展示了基于常系数微分方程的特殊解法。
   - 提到了多次方程 \(y'' - p(x)y' - q(x)y = f(x)\) 的解的特殊形式。

3. **第二部分**：
   - **二阶及高阶微分方程**：例如 \( y'' + p(x)y' + q(x)y = 0 \)，常常需要求解对应的齐次方程或非齐次方程。
   - 高阶微分方程的求解，展示了使用特征方程的方法以及非齐次方程的特解形式。

图中的公式和处理方法是解决微分方程常用的数学工具，用于物理、工程及其他科学领域中模型的数学建模。具体每个公式的应用和解析方法在高等数学或工程数学教程中会有详细讲解。